In [3]:
import json
import re
import logging
from typing import Any, Dict, Optional

logger = logging.getLogger(__name__)

def parse_json_safely(text: str) -> Optional[Dict[str, Any]]:
    """
    Attempts to parse JSON from the given text using multiple methods.
    
    Args:
    text (str): The text to parse JSON from.
    
    Returns:
    Optional[Dict[str, Any]]: Parsed JSON object or None if parsing fails.
    """
    logger.debug(f"Attempting to parse JSON from: {text}")

    # Method 1: Direct JSON parsing
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        logger.debug(f"Initial JSON parse failed: {e}")

    # Method 2: Regex-based JSON-like structure extraction
    json_like_pattern = r'\{(?:[^{}]|(?R))*\}'
    matches = re.findall(json_like_pattern, text)
    for match in matches:
        try:
            return json.loads(match)
        except json.JSONDecodeError:
            continue

    # Method 3: Line-by-line parsing
    lines = text.split('\n')
    accumulated = ""
    for line in lines:
        accumulated += line
        try:
            return json.loads(accumulated)
        except json.JSONDecodeError:
            continue

    # Method 4: Manual key-value extraction
    manual_parse = {}
    key_value_pattern = r'"(\w+)"\s*:\s*([^,}\s]+|"[^"]*")'
    matches = re.finditer(key_value_pattern, text)
    for match in matches:
        key, value = match.groups()
        try:
            parsed_value = json.loads(value)
        except json.JSONDecodeError:
            parsed_value = value.strip('"')
        manual_parse[key] = parsed_value

    if manual_parse:
        logger.info(f"Manually parsed JSON: {manual_parse}")
        return manual_parse

    logger.warning("Failed to parse any JSON-like structure")
    return None

def safe_json_parser(func):
    """
    Decorator to safely parse JSON output from a function.
    
    Args:
    func: The function to decorate.
    
    Returns:
    A wrapper function that safely parses JSON.
    """
    async def wrapper(*args, **kwargs):
        result = await func(*args, **kwargs)
        if isinstance(result, str):
            parsed = parse_json_safely(result)
            if parsed is not None:
                return parsed
            else:
                logger.warning(f"Failed to parse JSON from function {func.__name__}")
                return result  # Return original string if parsing fails
        return result
    return wrapper


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


ModuleNotFoundError: No module named 'langchain.schema.runnable'; 'langchain.schema' is not a package